In [48]:
import findspark
findspark.init()

In [49]:
from pyspark.sql import SparkSession

spark = SparkSession.builder. \
    appName("pyspark-1"). \
    getOrCreate()

### Read data

In [122]:
from pyspark.sql.functions import col, year, to_timestamp
source_df = spark.read.csv("/dataset/nyc-jobs.csv", header=True,quote='"',inferSchema=False,escape='"',sep=",")
source_df=source_df.withColumn( "Posting_Year", year(to_timestamp(col("Posting Date"), "yyyy-MM-dd'T'HH:mm:ss.SSS")) )
source_df.printSchema()

root
 |-- Job ID: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Posting Type: string (nullable = true)
 |-- # Of Positions: string (nullable = true)
 |-- Business Title: string (nullable = true)
 |-- Civil Service Title: string (nullable = true)
 |-- Title Code No: string (nullable = true)
 |-- Level: string (nullable = true)
 |-- Job Category: string (nullable = true)
 |-- Full-Time/Part-Time indicator: string (nullable = true)
 |-- Salary Range From: string (nullable = true)
 |-- Salary Range To: string (nullable = true)
 |-- Salary Frequency: string (nullable = true)
 |-- Work Location: string (nullable = true)
 |-- Division/Work Unit: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Minimum Qual Requirements: string (nullable = true)
 |-- Preferred Skills: string (nullable = true)
 |-- Additional Information: string (nullable = true)
 |-- To Apply: string (nullable = true)
 |-- Hours/Shift: string (nullable = true)
 |-- Work Locatio

In [105]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import trim, initcap
df = source_df.fillna({"Job Category": "Unknown","Salary Frequency": "Unknown","# Of Positions": 0})
df = df.withColumn("Job Category", initcap(trim(col("Job Category"))))
df = df.withColumn("Salary Frequency", initcap(trim(col("Salary Frequency"))))
df = df.withColumn("# Of Positions", col("# Of Positions").cast("int"))
df = (df.withColumn("Salary Range From", col("Salary Range From").cast("double")) 
    .withColumn("Salary Range To", col("Salary Range To").cast("double")))

### Sample function

In [106]:
def get_salary_frequency(df: DataFrame) -> list: 
    row_list = df.select('Salary Frequency').distinct().orderBy('Salary Frequency').collect() 
    return [row['Salary Frequency'] for row in row_list]

### Example of test function

In [107]:
mock_data = [('A', 'Annual'), ('B', 'Daily')]
expected_result = ['Annual', 'Daily']

In [108]:
def test_get_salary_frequency(mock_data: list, 
                              expected_result: list,
                              schema: list = ['id', 'Salary Frequency']):  
    mock_df = spark.createDataFrame(data = mock_data, schema = schema)
    assert get_salary_frequency(mock_df) == expected_result

In [109]:
#Whats the number of jobs posting per category (Top 10)
from pyspark.sql import Window
from pyspark.sql.functions import sum, col, dense_rank, desc

# Define window partitioned by nothing (global) and ordered by total positions
windowSpec = Window.orderBy(desc("Total Positions"))

# Aggregate positions per category
job_counts = (df.groupBy("Job Category") 
               .agg(sum(col("# Of Positions")).alias("Total Positions")))

# Apply ranking
ranked_jobs = job_counts.withColumn("rank", dense_rank().over(windowSpec))

# Show top 10 categories
ranked_jobs.filter(col("rank") <= 10).show(truncate=False)


+-----------------------------------------+---------------+----+
|Job Category                             |Total Positions|rank|
+-----------------------------------------+---------------+----+
|Public Safety, Inspections, & Enforcement|1407           |1   |
|Building Operations & Maintenance        |1249           |2   |
|Engineering, Architecture, & Planning    |762            |3   |
|Legal Affairs                            |515            |4   |
|Technology, Data & Innovation            |405            |5   |
|Health                                   |358            |6   |
|Administration & Human Resources         |330            |7   |
|Finance, Accounting, & Procurement       |275            |8   |
|Maintenance & Operations                 |212            |9   |
|Policy, Research & Analysis              |200            |10  |
+-----------------------------------------+---------------+----+



In [110]:
from pyspark.sql.functions import when, col,avg, min, max 
#Whats the salary distribution per job category?
df = (df.withColumn(
    "Annual Salary From",
    when(col("Salary Frequency")=="Hourly", col("Salary Range From")*40*52)
    .when(col("Salary Frequency")=="Daily", col("Salary Range From")*260)
    .when(col("Salary Frequency")=="Weekly", col("Salary Range From")*52)
    .otherwise(col("Salary Range From"))))


df = (df.withColumn(
    "Annual Salary To",
    when(col("Salary Frequency")=="Hourly", col("Salary Range To")*40*52)
    .when(col("Salary Frequency")=="Daily", col("Salary Range To")*260)
    .when(col("Salary Frequency")=="Weekly", col("Salary Range To")*52)
    .otherwise(col("Salary Range To"))))


salary_distribution = (df.groupBy("Job Category")
.agg( avg(col("Annual Salary From")).alias("Avg Salary From"), avg(col("Annual Salary To")).alias("Avg Salary To"), min(col("Annual Salary From")).alias("Min Salary"), max(col("Annual Salary To")).alias("Max Salary") ) 
.orderBy(col("Avg Salary To").desc()))
salary_distribution.show(10, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+-------------+----------+----------+
|Job Category                                                                                                                                                                                             |Avg Salary From|Avg Salary To|Min Salary|Max Salary|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+-------------+----------+----------+
|Administration & Human Resources Finance, Accounting, & Procurement Building Operations & Maintenance                                                                                                    |218587.0       |218587.0     

In [111]:
from pyspark.sql.functions import col, max, when
#Whats the job posting having the highest salary per agency?
# Normalize salary to annual equivalent
df_norm = (source_df.withColumn("Salary_To", col("Salary Range To").cast("double"))
        .withColumn("Salary_Normalized",
        when(col("Salary Frequency") == "Annual", col("Salary_To"))
        .when(col("Salary Frequency") == "Daily", col("Salary_To") * 260)
        .when(col("Salary Frequency") == "Weekly", col("Salary_To") * 52)
        .when(col("Salary Frequency") == "Hourly", col("Salary_To") * 2080)
        .otherwise(None))
           .select("Agency","Business Title","Job Category","Salary Frequency","Salary Range From",
             "Salary Range To","Salary_Normalized"))

# Highest salary per agency
highest_salary_per_agency = (df_norm.groupBy("Agency") 
    .agg(max("Salary_Normalized").alias("Max_Salary")).select("Agency","Max_Salary"))

# Alias both DataFrames
jobs = df_norm.alias("jobs")
maxsal = highest_salary_per_agency.alias("maxsal")

# Join back to get job posting details
highest_jobs = (
    maxsal.join(jobs,
    (maxsal.Agency == jobs.Agency) & (maxsal.Max_Salary == jobs.Salary_Normalized),
    "inner")).selectExpr("maxsal.Agency","maxsal.Max_Salary","jobs.`Business Title` as Job_Posting",
                        "jobs.`Job Category`", "jobs.`Salary Frequency`",
                         "jobs.`Salary Range From`","jobs.`Salary Range To`")

highest_jobs.show(truncate=False)


+------------------------------+----------+------------------------------------------------+-----------------------------------------------------------------------------------------------------+----------------+-----------------+---------------+
|Agency                        |Max_Salary|Job_Posting                                     |Job Category                                                                                         |Salary Frequency|Salary Range From|Salary Range To|
+------------------------------+----------+------------------------------------------------+-----------------------------------------------------------------------------------------------------+----------------+-----------------+---------------+
|DEPARTMENT OF BUSINESS SERV.  |162014.0  |EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT        |null                                                                                                 |Annual          |60740            |162014         |
|OFFICE OF COLLE

In [128]:
#Whats the job positings average salary per agency for the last 2 years?
from pyspark.sql.functions import col, avg, count, year, when, current_date

# Step 1: Normalize salary to annual equivalent
df_norm = (source_df.withColumn("Salary_To", col("Salary Range To").cast("double"))
    .withColumn("Salary_Normalized",
        when(col("Salary Frequency") == "Annual", col("Salary_To"))
        .when(col("Salary Frequency") == "Daily", col("Salary_To") * 260)
        .when(col("Salary Frequency") == "Weekly", col("Salary_To") * 52)
        .when(col("Salary Frequency") == "Hourly", col("Salary_To") * 2080)
        .otherwise(None))).select("Job ID","Agency","Salary_Normalized","Posting Date","Posting_Year")

# Step 2: Extract posting year
from pyspark.sql.functions import col, year, to_date,current_date

current_year = year(current_date())
df_recent = df_norm.filter(col("Posting_Year") >= (current_year - 2))
avg_salary_per_agency =(df_recent.groupBy("Agency") 
    .agg(avg("Salary_Normalized").alias("Avg_Salary"), count("*").alias("Num_Postings")))
avg_salary_per_agency.orderBy(col("Avg_Salary").desc()).show(truncate=False)


+------+----------+------------+
|Agency|Avg_Salary|Num_Postings|
+------+----------+------------+
+------+----------+------------+



In [132]:
from pyspark.sql.functions import col, max, avg, when, explode, split

# Step 1: Normalize salary
df_norm = source_df.withColumn("Salary_To", col("Salary Range To").cast("double")) \
    .withColumn("Salary_Normalized",
        when(col("Salary Frequency") == "Annual", col("Salary_To")) \
        .when(col("Salary Frequency") == "Daily", col("Salary_To") * 260) \
        .when(col("Salary Frequency") == "Weekly", col("Salary_To") * 52) \
        .when(col("Salary Frequency") == "Hourly", col("Salary_To") * 2080) \
        .otherwise(None)
    )

# Step 2: Split Preferred Skills into individual skills
df_skills = df_norm.withColumn("Skill", explode(split(col("Preferred Skills"), "•|,|;"))).select("Skill").distinct()
df_skills.show(truncate=False)



+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Skill                                                                                                                                                                                                                        |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| and to maneuver through confined spaces.  The candidate must be able to effectively communicate both verbally and in writing.  A valid motor vehicle driverâ€™s license is required.                                        |
|â€¢	Able to lift 50 pounds without restrictions.  â€¢	Strong oral & written communication skills. â€¢	A